In [149]:
import pandas as pd
import os, errno
import time
import itertools

In [150]:
meta_data = pd.read_csv('~/vox1_meta.csv', sep='\t')

In [151]:
#print(meta_data["Nationality"].value_counts())
# print(meta_data["Gender"].value_counts())

In [152]:
# meta_data

In [153]:
def get_ids(data, keys, values):
    for key_idx, value in enumerate(values):
        # set data to subset with value for given key
        data = data[data[keys[key_idx]] == value]
        
    return set(data['VoxCeleb1 ID'])

# @brief Return list of tuples of all combinations of keys in
#        the sensitive param provided as args
def get_intersection_combos(*sensitive_param_value_groups):
    return list(itertools.product(*sensitive_param_value_groups))

In [154]:
keys = ['Gender', 'Nationality']
gender_values = ['m', 'f']
# @TODO Add all countries
#country_values = ['USA', 'UK', 'Canada']
country_values = ['Australia', 'India', 'Norway', 'Ireland', 'Germany', 'New Zealand', 'Italy','Mexico',\
                  'Sweden', 'Spain', 'Russia', 'Switzerland', 'Chile', 'Philippines', 'Croatia', 'Denmark',\
                  'Netherlands', 'Poland', 'Portugal', 'China', 'France', 'Guyana', 'Singapore', 'Brazil',\
                  'Sri Lanka', 'South Africa', 'South Korea', 'Trinidad and Tobago', 'Pakistan', 'Austria',\
                  'Israel', 'Iran', 'Sudan']

intersections_vals = get_intersection_combos(gender_values, country_values)
intersections_ids = []

for intersection in intersections_vals:
    intersections_ids.append(get_ids(meta_data, keys, intersection))
    print(f"Intersection {intersection}.count -> {len(intersections_ids[-1])}")

Intersection ('m', 'Australia').count -> 25
Intersection ('m', 'India').count -> 15
Intersection ('m', 'Norway').count -> 13
Intersection ('m', 'Ireland').count -> 13
Intersection ('m', 'Germany').count -> 4
Intersection ('m', 'New Zealand').count -> 6
Intersection ('m', 'Italy').count -> 3
Intersection ('m', 'Mexico').count -> 5
Intersection ('m', 'Sweden').count -> 1
Intersection ('m', 'Spain').count -> 2
Intersection ('m', 'Russia').count -> 0
Intersection ('m', 'Switzerland').count -> 3
Intersection ('m', 'Chile').count -> 2
Intersection ('m', 'Philippines').count -> 1
Intersection ('m', 'Croatia').count -> 2
Intersection ('m', 'Denmark').count -> 1
Intersection ('m', 'Netherlands').count -> 1
Intersection ('m', 'Poland').count -> 0
Intersection ('m', 'Portugal').count -> 1
Intersection ('m', 'China').count -> 1
Intersection ('m', 'France').count -> 0
Intersection ('m', 'Guyana').count -> 0
Intersection ('m', 'Singapore').count -> 1
Intersection ('m', 'Brazil').count -> 0
Intersect

In [155]:
# load all utternaces
utters = pd.read_csv('/home/jupyter/voxceleb-fairness/data/datasets/full/vox1_full_utterances.txt', header=None, sep=' ')
intersections_utters = []

# extract all utterance subsets
for idx, intersection_ids in enumerate(intersections_ids):
    intersection_utters = utters[[x in intersection_ids for x in utters[0]]]
    sample_len = min(len(intersection_utters), 5000)
    intersections_utters.append(intersection_utters.sample(sample_len).reset_index(drop=True))
    # print(intersections_vals[idx])
    # print(intersections_utters[-1])

In [156]:
# create list files from utterance subset files

def silentremove(filename):
    try:
        os.remove(filename)
    except OSError as e: # this would be "except OSError, e:" before Python 2.6
        if e.errno != errno.ENOENT: # errno.ENOENT = no such file or directory
            raise # re-raise exception if a different error occurred
            
def make_pairs(utters_df, filename):
    silentremove(filename)
    start = time.time()
    current_iter_start = start
    for i in range(len(utters_df)):
        data = []
        for j in range(i + 1, len(utters_df)):
            num = 1 if utters_df[0][i] == utters_df[0][j] else 0
            data.append([num, utters_df[1][i], utters_df[1][j]])
        pd.DataFrame(data).to_csv(filename, mode='a', index=False, header=None, sep=' ')
        if i % 1000 == 0:
            current_iter_end = time.time()
            print('Wrote {} of {} utterances in {} seconds ({} seconds from start)'.format(i, len(utters_df), current_iter_end - current_iter_start, current_iter_end - start))
            current_iter_start = current_iter_end

def clean_intersection_name(name):
    name = name.replace(')','')
    name = name.replace('(','')
    name = name.replace('\'','')
    name = name.replace(' ','')
    name = name.replace(',','_')
    return name

In [157]:
# extract the full lists
list_fnames = []
for idx, intersection_utters in enumerate(intersections_utters):
    # get file name for list and make dirs
    name = clean_intersection_name(str(intersections_vals[idx]))
    list_fnames.append(f"/home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_{name}.txt")
    os.makedirs(os.path.dirname(list_fnames[-1]), exist_ok=True)
    # write the list
    make_pairs(intersection_utters, list_fnames[-1])

Wrote 0 of 3157 utterances in 0.18897175788879395 seconds (0.18897175788879395 seconds from start)
Wrote 1000 of 3157 utterances in 156.11747312545776 seconds (156.30644488334656 seconds from start)
Wrote 2000 of 3157 utterances in 96.4682207107544 seconds (252.77466559410095 seconds from start)
Wrote 3000 of 3157 utterances in 39.568827867507935 seconds (292.3434934616089 seconds from start)
Wrote 0 of 3034 utterances in 0.17218542098999023 seconds (0.17218542098999023 seconds from start)
Wrote 1000 of 3034 utterances in 148.18790888786316 seconds (148.36009430885315 seconds from start)
Wrote 2000 of 3034 utterances in 89.34569311141968 seconds (237.70578742027283 seconds from start)
Wrote 3000 of 3034 utterances in 32.033486127853394 seconds (269.7392735481262 seconds from start)
Wrote 0 of 1744 utterances in 0.10055708885192871 seconds (0.10055708885192871 seconds from start)
Wrote 1000 of 1744 utterances in 73.05301356315613 seconds (73.15357065200806 seconds from start)
Wrote 0 of

In [164]:
# create the balanced lists with equal positive and negative test pairs
def balance_pairs(unbalanced_pairs):
    return pd.concat([unbalanced_pairs[unbalanced_pairs[0] == 1].reset_index(drop=True),\
               unbalanced_pairs[unbalanced_pairs[0] == 0].sample(sum(unbalanced_pairs[0] == 1)).\
               reset_index(drop=True)]).sort_index().reset_index(drop=True)

for list_fname in list_fnames:
    try:
        print(f"Running {list_fname}")
        # set new file name
        new_fname = list_fname.replace('.txt', '_balanced.txt')
        # read in old data and etract balanced list
        pairs = pd.read_csv(list_fname, header=None, sep=' ')
        balanced_pairs = balance_pairs(pairs)
        # write the new list
        balanced_pairs.to_csv(new_fname, index=False, header=None, sep=' ')
    except ValueError:
        print(f">> File is empty... skipping")
    except FileNotFoundError:
        print(f">> File not found... skipping")

Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Australia.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_India.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Norway.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Ireland.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Germany.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_NewZealand.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Italy.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Mexico.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Sweden.txt
File is empty... skipping
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_Spain.txt
Running /home/jupyter/voxceleb-fairness/data/lists/intersect/vox1_intersect_m_